<a href="https://colab.research.google.com/github/AngelTroncoso/ML_Alura/blob/main/4_Regresion_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Precificación inmobiliaria**

**Objetivo**: Estimar los precios de los inmuebles.
 * Identificar los aspectos que tienen más influencia en la precificación de los inmuebles.
 * Entender cuál de todos estos aspectos es el más relevante.
 * Precificar un inmueble nuevo.

Base de datos simplificada e inspirada en [House Prices](https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques)



![](https://i.imgur.com/A8X79yq.jpeg)


# 1. Ajustando una recta

## Conociendo los datos

In [17]:
from google.colab import files

# Sube el archivo manualmente
uploaded = files.upload()

# Cargar los datos desde el archivo subido
datos = pd.read_csv(next(iter(uploaded.keys())), sep=',', encoding='latin1')
print(datos.head())


Saving precios_casas.csv to precios_casas.csv
   Id  area_primer_piso  tiene_segundo_piso  area_segundo_piso  \
0   1           79.5224                   1            79.3366   
1   2          117.2398                   0             0.0000   
2   3           85.4680                   1            80.4514   
3   4           89.2769                   1            70.2324   
4   5          106.3705                   1            97.8237   

   cantidad_banos  capacidad_carros_garage  calidad_de_cocina_excelente  \
0               2                      548                            0   
1               2                      460                            0   
2               2                      608                            0   
3               1                      642                            0   
4               2                      836                            0   

   precio_de_venta  
0        1027905.0  
1         894795.0  
2        1101855.0  
3         690200.0  
4

In [ ]:
#Cuáles factores fueron colectados


## Correlación

**¿Cuáles factores están relacionados con el precio de la casa? ¿Cómo es esa relación?**

El coeficiente de correlación de Pearson nos permite medir la relación lineal entre variables, ofreciendo una escala que varía de -1 a 1, la cual interpretamos según su intensidad y dirección:

* **-1**: correlación negativa perfecta: a medida que una variable aumenta, la otra disminuye.  
* **0**: no hay relación lineal entre las variables.  
* **1**: correlación positiva perfecta: a medida que una variable aumenta, la otra también aumenta.  



In [ ]:
# Correlación


In [ ]:
# Actividad
# Cuáles factores están más correlacionados?


## Relacionando variables

In [ ]:
# importando los paquetes para la visualización


In [ ]:
# Cómo se relacionan el área construída y el precio del inmueble?


In [ ]:
# Aparentemente mientras mayor es el área del primer piso, mayor es el precio de la casa.
# E si quisieramos trazar una línea que nos permita describir este comportamiento?


## Mejor recta

In [ ]:
# Cuál es la recta que mejor se ajusta a esta relación


# 2. Explicando la recta
Ajustamos una recta entre los $m^2$ del primer piso y el precio de la casa. Queremos explicar el precio de la casa a partir de su tamaño, por eso decimos que:

* Variable explicativa/independiente: Área del primer piso  
* Variable de respuesta/dependiente: Precio de la casa  

In [ ]:
#Quién es nuestra variable de respuesta?


### Separando datos para entrenamiento y para prueba

El conjunto de **entrenamiento** se utiliza para ajustar el modelo, mientras que el conjunto de **prueba** se usa para evaluar su desempeño al predecir precios de viviendas no vistos durante el entrenamiento, lo que ayuda a la generalización del modelo.

In [ ]:
# import train_test_split


In [ ]:
#Datos de entrenamiento para usar la fórmula


In [ ]:
# import ols


In [ ]:
# ajustando el primer modelo


## Analizando los coeficientes

(intensidad, dirección y significancia)




In [ ]:
# visualizando los parámetros


In [ ]:
# El resumen del modelo


- El **punto de intercepción** es el valor esperado del precio de la casa cuando todas las demás variables son cero. En este caso, cuando todas las demás variables, incluida el área del primer piso, son cero, el precio esperado de la casa es de $145.196,40. No siempre se tiene una interpretación práctica para este número.

- El **efecto individual** del área es de \$6.833,97. Esto indica que por cada metro cuadrado adicional al área del primer piso, se espera que el precio de la casa aumente en promedio \$6.833,97.

<img src="https://i.ibb.co/CJwfMgW/coeficientes.png" alt="coeficientes" border="0">


## Explicabilidad del modelo

**¿En qué medida la variación del área explica los diversos precios de las casas?**  

En este caso, recurrimos a la métrica R², el coeficiente de determinación. El R² varía de 0 a 1, donde 1 indica un ajuste perfecto del modelo a los datos, es decir, todas las variaciones en la variable dependiente son explicadas por las variables independientes en el modelo. Por otro lado, un R² de 0 indica que el modelo no explica ninguna variabilidad en la variable dependiente.

In [ ]:
# observando el R²


## Entendiendo el resíduo

In [ ]:
# Quiénes son los residuos?


In [ ]:
# Cómo están distribuidos


<img src="https://i.ibb.co/zGTB35V/residuos.png" alt="residuos" border="0">

## Obteniendo el R² de la previsión

In [ ]:
# definiendo la 'Y' prevista


In [ ]:
# importando el r2_score


In [ ]:
# mostrando el resultado de r²


# 3. Añadiendo otras características

El modelo con un solo factor nos mostró un R² de 0.38, es decir, aproximadamente el 38% de la variación observada en los precios de las casas puede explicarse por la variación en el área.  
Esto indica que aún existe una cantidad significativa de variación que no está siendo capturada por este modelo específico. Analizaremos otros factores para explicar el precio de las casas.

## Analizando los factores

In [ ]:
# cuáles otras características podrían explicar el precio de los inmuebles?


In [ ]:
#Vamos a analizar solamente y_vars='precio_de_venta'


## Añadiendo factores al modelo

In [ ]:
# importando la api de statsmodels


In [ ]:
# Añadiendo una constante


In [ ]:
# Creando el modelo de regresión (con todas las variables): saturado


In [ ]:
# Modelo sin el área del segundo piso


In [ ]:
# Modelo sin la información del garage
# Modelo sin el área del segundo piso


In [ ]:
# Resumen del modelos


## Comparando los modelos
Cuál es el modelo?


In [ ]:
# Cuántos parámetros tiene el modelo?


# 4. Precificando las casas

## Obteniendo el R² de la previsión

In [ ]:
# Añadiendo una constante en X_test


In [ ]:
# Previsión con el modelo 3


In [ ]:
# Cuál es el r² de la previsión?


In [ ]:
# Cuál es el R² del entrenamiento?


## Precificando una casa

<img src="https://i.ibb.co/SvG4V4j/costo.png" alt="costo" border="0">

In [ ]:
#Nuevo inmueble


In [ ]:
# Cuál es el precio del inmueble de acuerdo con el modelo 0?


In [ ]:
#Cuál es el precio del inmueble de acuerdo con el modelo 3?


## Precificando varias casas

<img src="https://i.imgur.com/i0skFlB.jpeg" width="600"/>

In [ ]:
# Leyendo varias casas?


In [ ]:
# Añadiendo una constante


In [ ]:
# Cuál es el precio de estas nuevas casas?
# precio_casas = pd.DataFrame()


# 5. Investigando el modelo





## Multicolinearidad

Cuando dos o más cosas que estás intentando estudiar son tan parecidas que es difícil determinar cuánto está influyendo cada una de ellas en el resultado.

In [ ]:
# importar el VIF de statsmodel


In [ ]:
# VIF 1


In [ ]:
# VIF 3


## Análisis de los residuos

Residuos: la discrepancia entre los valores reales y los valores que el modelo predice para los mismos puntos de datos.

<img src="https://i.ibb.co/9bmySbm/heterocedasticidad.png" alt="heterocedasticidad" border="0">